Iteration methods in parallel
===

In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)
c.ids

In [ ]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD

from ngsolve import *

from netgen.geom2d import unit_square
comm = MPI.COMM_WORLD
if comm.rank == 0:
    mesh = Mesh(unit_square.GenerateMesh(maxh=0.1).Distribute(comm))
else:
    mesh = Mesh(netgen.meshing.Mesh.Receive(comm))

setup a standard problem, in parallel. We use a Jacobi preconditioner: it extracts local diagonals, and cumulates them over identified dofs.

In [ ]:
%%px
fes = H1(mesh, dirichlet='.*')
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)

pre = Preconditioner(a, "local")
pre.Update()

The right hand side vector `f.vec` is distributed, the vector `gfu.vec` of the gridfunction is consistent. Create help vectors of the same type:

* residual calculation $r = f - A u$ is purely local
* the preconditioning step $w = pre * r$ includes type conversion from a distributed input to a consistent output
* the inner product for the error acts on two vectors of opposite type
* solution vector update can be done purely local

In [ ]:
%%px
r = f.vec.CreateVector()    # a distributed vector
w = gfu.vec.CreateVector()  # a consistent vector

for it in range(100):
    r.data = f.vec - a.mat * gfu.vec
    w.data = pre * r
    err = InnerProduct(w, r)
    if comm.rank==0: print (err)
    gfu.vec.data += w

In [ ]:
from ngsolve.webgui import Draw

gfu = c[:]['gfu']
Draw(gfu[0]);

Very similar for other iteraton methods, such as CG. The matrix operation goes from consistent to distributed without communication, the preconditioner does the cumulation. The inner products are between different vector types:

In [ ]:
%%px
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=comm.rank==0)

gfu.vec.data = inv * f.vec

In [ ]:
gfu = c[:]['gfu']
Draw(gfu[0]);